In [ ]:
#!pip install pandas

In [4]:
import sys  
sys.path.insert(1, './src/')

from train_tree import *


In [5]:
# generate pseudo data 
n_sample = 500
n_dim = 50
X_train = np.random.normal(size=(n_sample, n_dim))
Y_train = np.random.randint(2, size=n_sample)

In [6]:
train_data = convert_to_DMatrix(X_train, Y_train)
valid_data = convert_to_DMatrix(X_train, Y_train)

In [7]:
xgb_model = train_model(train_data, valid_data, xgb_params, xgb_option)

auc = evaluate_model(xgb_model, valid_data)

print(auc)

[0]	valid-auc:0.57785
[1]	valid-auc:0.60641
[2]	valid-auc:0.63189
[3]	valid-auc:0.66043
[4]	valid-auc:0.67723
[5]	valid-auc:0.69483
[6]	valid-auc:0.68607
[7]	valid-auc:0.69150
[8]	valid-auc:0.69782
[9]	valid-auc:0.69814
[10]	valid-auc:0.69945
[11]	valid-auc:0.71186
[12]	valid-auc:0.71009
[13]	valid-auc:0.71951
[14]	valid-auc:0.72026
[15]	valid-auc:0.72653
[16]	valid-auc:0.72516
[17]	valid-auc:0.72955
[18]	valid-auc:0.72685
[19]	valid-auc:0.73378
[20]	valid-auc:0.73080
[21]	valid-auc:0.73668
[22]	valid-auc:0.74147
[23]	valid-auc:0.74575
[24]	valid-auc:0.74569
[25]	valid-auc:0.74738
[26]	valid-auc:0.74654
[27]	valid-auc:0.75038
[28]	valid-auc:0.75073
[29]	valid-auc:0.75234
[30]	valid-auc:0.75294
[31]	valid-auc:0.75552
[32]	valid-auc:0.75951
[33]	valid-auc:0.75970
[34]	valid-auc:0.76092
[35]	valid-auc:0.76153
[36]	valid-auc:0.76185
[37]	valid-auc:0.76257
[38]	valid-auc:0.76257
[39]	valid-auc:0.76134
[40]	valid-auc:0.76182
[41]	valid-auc:0.76153
[42]	valid-auc:0.76211
[43]	valid-auc:0.7626

In [8]:
# save to file 
xgb_model.save_model('model/xgb_model.ckpt')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [20:46:53] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [9]:
# load model 
model_path = 'model/xgb_model.ckpt'
xgb_model = xgb.Booster()
xgb_model.load_model(model_path)
df_trees = xgb_model.trees_to_dataframe()

## Tree-regularized Embedding Part 

In [17]:
from tree_embedding import TreeToVector, TreeToVectorConverter, TreeToToken, TreeToTokenConverter

In [18]:
xgb_model_path = 'model/xgb_model.ckpt'

xgbTree = TreeToVectorConverter(xgb_model_path, X_train.shape[1])

In [23]:
import torchvision.transforms as transforms

transform_batch = transforms.Compose([TreeToVector(xgbTree)])

In [24]:
from torch.utils.data import Dataset, DataLoader

class ClassifierDataset(Dataset):
    def __init__(self, X_data, Y_data):
        self.X = X_data
        self.Y = Y_data

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_batch = self.X[idx]
        Y_batch = self.Y[idx]
        return X_batch, Y_batch

In [25]:
import torch 
import torch.nn as nn

train_data = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train).long())
    
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


In [27]:
for epoch in range(10):
    print(epoch+1)    
    for batch_idx, (data, target) in enumerate(train_loader):
        # transform 
        data = transform_batch(data)

1
2
3
4
5
6
7
8
9
10
